In [15]:
import time
import os
import csv
import numba
import math
from coffea.nanoevents import BaseSchema

import awkward as ak
import numpy as np
from coffea import processor, hist

from coffea.nanoevents.methods import candidate
ak.behavior.update(candidate.behavior)

import mplhep
import matplotlib.pyplot as plt
plt.style.use(mplhep.style.CMS)

# To import bot
import sys

sys.path.insert(1, '/afs/cern.ch/work/m/mabarros/public/CMSSW_10_6_12/src/OniaOpenCharmRun2ULAna/bot')

import bot_config as bot

In [16]:
muon_cols = ['Muon_charge', 'Muon_dxy', 'Muon_dxyErr', 'Muon_dz', 'Muon_dzErr', 'Muon_eta', 'Muon_isGlobal', 'Muon_mass',
             'Muon_phi', 'Muon_pt', 'Muon_ptErr', 'Muon_softId', 'Muon_vtxIdx', 'Muon_vtxFlag', 'Muon_simIdx']

dimu_cols = ['Dimu_pt', 'Dimu_eta', 'Dimu_phi', 'Dimu_rap', 'Dimu_mass', 'Dimu_charge', 'Dimu_vtxIdx', 'Dimu_chi2', 'Dimu_dl',
             'Dimu_dlErr', 'Dimu_dlSig', 'Dimu_cosphi', 'Dimu_x', 'Dimu_y', 'Dimu_z', 'Dimu_t1muIdx', 'Dimu_t2muIdx',]

d0_cols = ['D0_pt', 'D0_eta', 'D0_phi', 'D0_rap', 'D0_mass12', 'D0_mass21', 'D0_vtxIdx', 'D0_chi2', 'D0_dl', 'D0_dlErr', 'D0_dlSig',
           'D0_cosphi', 'D0_x', 'D0_y', 'D0_z', 'D0_hasMuon',
           'D0_t1pt', 'D0_t1eta', 'D0_t1phi', 'D0_t1chindof', 'D0_t1nValid', 'D0_t1nPix', 'D0_t1dxy', 'D0_t1dz', 'D0_t1chg', 
           'D0_t2pt', 'D0_t2eta', 'D0_t2phi', 'D0_t2chindof', 'D0_t2nValid', 'D0_t2nPix', 'D0_t2dxy', 'D0_t2dz', 'D0_t2chg',]

dstar_cols = ['Dstar_pt', 'Dstar_eta', 'Dstar_phi', 'Dstar_rap', 'Dstar_deltam', 'Dstar_deltamr', 'Dstar_vtxIdx', 'Dstar_hasMuon',
              'Dstar_D0pt', 'Dstar_D0eta', 'Dstar_D0phi', 'Dstar_D0mass', 'Dstar_D0chi2', 'Dstar_D0dl', 'Dstar_D0dlErr',
              'Dstar_D0dlSig', 'Dstar_D0cosphi', 'Dstar_D0x', 'Dstar_D0y', 'Dstar_D0z',
              'Dstar_Kpt', 'Dstar_Keta', 'Dstar_Kphi', 'Dstar_KvtxIdx', 'Dstar_Kchindof', 'Dstar_KnValid', 'Dstar_KnPix', 'Dstar_Kdxy',
              'Dstar_Kdz', 'Dstar_Kchg', 'Dstar_pipt', 'Dstar_pieta', 'Dstar_piphi', 'Dstar_pivtxIdx', 'Dstar_pichindof', 'Dstar_pinValid',
              'Dstar_pinPix', 'Dstar_pidxy', 'Dstar_pidz', 'Dstar_pichg', 'Dstar_pispt', 'Dstar_piseta', 'Dstar_pisphi', 'Dstar_pisvtxIdx',
              'Dstar_pischindof', 'Dstar_pisnValid', 'Dstar_pisnPix',
              'Dstar_pisdxy', 'Dstar_pisdz', 'Dstar_simIdx', 'Dstar_D0simIdx',
              'Dstar_pisisHighPurity', 'Dstar_piisHighPurity', 'Dstar_KisHighPurity']

gen_part_cols = ['GenPart_eta', 'GenPart_genPartIdxMother', 'GenPart_mass', 'GenPart_pdgId', "GenPart_phi", "GenPart_pt", 'GenPart_status',
                'GenPart_Id', 'GenPart_parpdgId', 'GenPart_sparpdgId', 'GenPart_numberOfDaughters', 'GenPart_nstchgdaug', 'GenPart_vx', 
                'GenPart_vy', 'GenPart_vz', 'GenPart_mvx', 'GenPart_mvy', 'GenPart_mvz', 'GenPart_recIdx']

hlt_cols_charm_2017 = ['HLT_Dimuon0_Jpsi', 'HLT_Dimuon20_Jpsi_Barrel_Seagulls' , 'HLT_Dimuon25_Jpsi']

D0_PDG_MASS = 1.864

def get_vars_dict(events, col_list):
    dict = {}
    col = ''
    for c in col_list:
        if c.startswith('Muon'):
            col = c[5:]
        elif c.startswith('Dimu'):
            col = c[4:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('D0'):
            col = c[2:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('Dstar'):
            col = c[5:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('PVtx'):
            col = c[5:]
        elif c.startswith("GenPart"):
            col = c[8:]
        elif c.startswith('HLT'):
            col = c[:]
        else:
            Exception('Not good!')

        if col == 'x' or col == 'y' or col == 'z':
            col = 'vtx_' + col

        if len(events[c]) == 0:
            dict[col] = np.array([])
        else:
            dict[col] = events[c]
    return dict

In [17]:
def build_4mom_string(candidate):
    return f'({candidate.pt:.2f}, {candidate.eta:.2f}, {candidate.phi:.2f}, {candidate.mass:.2f})'

def to_cartesian(cand):
    x = cand.pt*math.cos(cand.phi)
    y = cand.pt*math.sin(cand.phi)
    z = cand.pt*math.sinh(cand.eta)
    t = math.sqrt(x*x + y*y + z*z + cand.mass*cand.mass)

    return t, x, y, z

def sum_cand(cand1, cand2):
    t1, x1, y1, z1 = to_cartesian(cand1)
    t2, x2, y2, z2 = to_cartesian(cand2)

    tr = t1+t2
    xr = x1+x2
    yr = y1+y2
    zr = z1+z2

    r = math.sqrt(xr*xr + yr*yr + zr*zr)
    pt = math.sqrt(xr*xr + yr*yr)
    eta = math.asinh(zr / r)
    phi = math.atan2(yr, xr)
    mass = math.sqrt(tr*tr - xr*xr - yr*yr - zr*zr)

    return f'({pt:.2f},{eta:.2f},{phi:.2f},{mass:.2f})'


def print_candidate(candidate):
    quadrimom = build_4mom_string(candidate)
    print(f"PDGId: {candidate.pdgId}, motherId: {candidate.genPartIdxMother}, n_daughters: {candidate.numberOfDaughters}, status: {candidate.status}, 4-momentum (pt, eta, phi, mass): {quadrimom}")

In [18]:
import re

files = []
#pth0 = "/eos/user/m/mabarros/Monte_Carlo/2017/0000"
pth0 = "/eos/user/m/mabarros/Monte_Carlo/2017_improved_cuts/0000"
with os.scandir(pth0) as it:
    for file in it:
        if file.name.endswith('.root') and (file.stat().st_size != 0):
            files.append(file.path)
#pth1 = "/eos/user/m/mabarros/Monte_Carlo/2017/0001"
pth1 = "/eos/user/m/mabarros/Monte_Carlo/2017_improved_cuts/0001"
with os.scandir(pth1) as it:
    for file in it:
        if file.name.endswith('.root') and (file.stat().st_size != 0):
            files.append(file.path)

            
            
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
            
files.sort(key=natural_keys)

In [19]:
def ratio_plot(hist_reco, hist_mc):
    fill_opts = {
    'edgecolor': (0,0,0,0.3),
    'alpha': 0.8
    }
    error_opts_1 = {
        'facecolor': 'none',
        'edgecolor': (0,0,0,.5),
        'linewidth': 0
    }
    data_err_opts = {
        'linestyle': 'none',
        'marker': '.',
        'markersize': 10.,
        'color': 'k',
        'elinewidth': 1,
    }
    
    error_opts_2 = {
        'facecolor': 'none',
        'linewidth': 0,
    }
    
    fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True
    )
    fig.subplots_adjust(hspace=.07)
    
    # plot the MC first
    hist.plot1d(
        hist_mc,
        ax=ax,
        clear=False,
        stack=False,
        line_opts=None,
        fill_opts=fill_opts,
        error_opts=error_opts_1
    )
    hist.plot1d(
        hist_reco,
        ax=ax,
        clear=False,
        stack=False,
        line_opts=None,
        fill_opts=fill_opts,
        error_opts=error_opts_2
    )
    
    ax.autoscale(axis='x', tight=True)
    ax.set_xlabel("")
    leg = ax.legend()
    
    hist.plotratio(
    num=hist_reco,
    denom=hist_mc,
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
    )
    rax.set_ylabel('Ratio')
    rax.set_ylim(0,2)
    
    return ax, rax

In [20]:
def create_plot1d(hist1d, log=False, ax=None):
    lumi = "x"
    
    from matplotlib.offsetbox import AnchoredOffsetbox, TextArea
    plt.style.use(mplhep.style.CMS)
    plt.rcParams.update({
        'font.size': 16,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 14,
        'ytick.labelsize': 14
    })
    fill_opts = {
    'alpha': 0.8,
    'edgecolor':(0,0,0,.5)
    }
    
    data_err_opts = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'k',
    'elinewidth': 1,}

    
    ax = hist.plot1d(hist1d, ax=ax)#, error_opts=data_err_opts)
    #mplhep.cms.label()
    
    
    # CMS format
    hfont = {'fontname':'Helvetica'}    
    plt.text(0.17, 0.89, "CMS", fontdict=hfont,  fontweight='bold', fontsize = 20, transform=plt.gcf().transFigure)
    plt.text(0.25, 0.89, "Simulation", fontdict=hfont, style='italic',fontsize = 20, transform=plt.gcf().transFigure)
    '''CMS = plt.text(0.03, 0.93, "CMS",
                  fontsize=22,
                  fontweight='bold',
                  horizontalalignment='left',
                  verticalalignment='bottom',
                  transform=ax.transAxes
                 )
    pre = plt.text(0.03, 0.88, "Preliminary",
                  fontsize=19,
                  fontstyle='italic',
                  horizontalalignment='left',
                  verticalalignment='bottom',
                  transform=ax.transAxes
                 )
    lumi = plt.text(1., 1., r"" + lumi +  "fb$^{-1}$ (13 TeV)",
                fontsize=18,
                fontstyle='normal',
                horizontalalignment='right',
                verticalalignment='bottom',
                transform=ax.transAxes
               )'''

    if log:
        ax.set_yscale('log')
        ax.set_ylim(1, None)
    else:
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,3), useMathText=True)
    
    axis = hist1d.axes()[0]
    centers = axis.centers()
    values = np.where(hist1d.values().get(()) < 0, 0, hist1d.values().get(()))
    
    # compute mean and std:
    mean = np.sum(values*centers)/np.sum(values)
    std = np.sqrt(np.sum(values*((centers - mean)**2))/np.sum(values))
    
    annotation = TextArea(f"Total: {np.sum(values):.2e}" \
                    + "\n" + f"Mean: {mean:.2e}" \
                    + "\n" + f"Std: {std:.2e}", textprops=dict(size=14))
    
    at = AnchoredOffsetbox('upper right', child=annotation)
    at.patch.set_facecolor('None')
    ax.add_artist(at)
    
    ax.legend().remove()
    
    return ax

In [21]:
def plot_2d(hist_2d):
    
    # CMS style
    import mplhep
    import matplotlib.pyplot as plt
    
    plt.style.use(mplhep.style.CMS)
    
    # Create figure
    fig = plt.figure()
    ax = fig.add_subplot()
    
    # plot
    hist.plot2d(hist_2d, xaxis='dstar_pt', ax=ax)
    mplhep.cms.label(ax=ax)
    

In [22]:
def association(cand1, cand2):
    ''' Function for association of the particles. The cuts that operates on all of them and 
    computation of quantities can go here. individual cuts can go on the main processing'''
    
    asso = ak.cartesian([cand1, cand2])
    asso = asso[asso.slot0.slot0.vtxIdx == asso.slot1.slot0.vtxIdx]
    asso = asso[ak.num(asso) > 0]
    
    cand1 = ak.zip({
            'pt': asso.slot0.slot0.pt,
            'eta': asso.slot0.slot0.eta,
            'phi': asso.slot0.slot0.phi,
            'mass': asso.slot0.slot0.mass,
            'charge': asso.slot0.slot0.charge}, with_name="PtEtaPhiMCandidate")
    
    gen_cand1 = ak.zip({
            'pt': asso.slot0.slot1.pt,
            'eta': asso.slot0.slot1.eta,
            'phi': asso.slot0.slot1.phi,
            'mass': asso.slot0.slot1.mass,
            'charge': asso.slot0.slot0.charge}, with_name="PtEtaPhiMCandidate")

    cand2 = ak.zip({
            'pt': asso.slot1.slot0.pt,
            'eta': asso.slot1.slot0.eta,
            'phi': asso.slot1.slot0.phi,
            'mass': asso.slot1.slot0.mass,
            'charge': asso.slot1.slot0.charge}, with_name="PtEtaPhiMCandidate")
    
    gen_cand2 = ak.zip({
            'pt': asso.slot1.slot1.pt,
            'eta': asso.slot1.slot1.eta,
            'phi': asso.slot1.slot1.phi,
            'mass': asso.slot1.slot1.mass,
            'charge': asso.slot1.slot0.charge}, with_name="PtEtaPhiMCandidate")
    
    cand1['rap'] = np.log((cand1.t+cand1.z)/(cand1.t-cand1.z))/2.
    cand2['rap'] = np.log((cand2.t+cand2.z)/(cand2.t-cand2.z))/2.
    gen_cand1['rap'] = np.log((gen_cand1.t+gen_cand1.z)/(gen_cand1.t-gen_cand1.z))/2.
    gen_cand2['rap'] = np.log((gen_cand2.t+gen_cand2.z)/(gen_cand2.t-gen_cand2.z))/2.
    
    '''if len(cand1) > 0:
        print(f'{cand1.t}, {cand1.z}')
        print(f"{asso.slot0.slot0.rap}, {cand1['rap']}, {gen_cand1['rap']}")
        print(f'{cand2.t}, {cand2.z}')
        print(f"{asso.slot1.slot0.rap}, {cand2['rap']}, {gen_cand2['rap']}")
        print("---------------------------------")'''
    
    
    asso['deltarap'] = cand1.rap - cand2.rap
    asso['deltapt'] = cand1.pt - cand2.pt
    asso['deltaeta'] = cand1.eta - cand2.eta
    asso['deltaphi'] = cand1.phi - cand2.phi
    
    asso['gen_deltarap'] = gen_cand1.rap - gen_cand2.rap
    asso['gen_deltapt'] = gen_cand1.pt - gen_cand2.pt
    asso['gen_deltaeta'] = gen_cand1.eta - gen_cand2.eta
    asso['gen_deltaphi'] = gen_cand1.phi - gen_cand2.phi
    
    asso['cand'] = cand1 + cand2
    asso['gen_cand'] = gen_cand1 + gen_cand2
    
    return asso

In [23]:
class GenTestProcessor(processor.ProcessorABC):
    def __init__(self):
        self._accumulator = processor.dict_accumulator({
            'cutflow': processor.defaultdict_accumulator(int),
            'n_matched_dimu': processor.value_accumulator(int),
            'n_reco_dimu': processor.value_accumulator(int),
            'n_gen_dimu': processor.value_accumulator(int),
            'n_pass_dimu': processor.value_accumulator(int),
            'n_double_counting_dimu': processor.value_accumulator(int),
            'n_matched_muon': processor.value_accumulator(int),
            'n_reco_muon': processor.value_accumulator(int),
            'n_gen_muon': processor.value_accumulator(int),
            'n_pass_muon': processor.value_accumulator(int),
            'n_matched_dstar': processor.value_accumulator(int),
            'n_reco_dstar': processor.value_accumulator(int),
            'n_pass_dstar': processor.value_accumulator(int),
            'n_gen_dstar': processor.value_accumulator(int),
            'n_double_counting_dstar': processor.value_accumulator(int),
            'matched_dimu_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,J/\Psi}$ matched [GeV]", 40, 0, 100)),
            'matched_dimu_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{J/\Psi} matched$", 60, -4, 4)),
            'matched_dimu_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{J/\Psi} matched [rad]$", 60, -3.5, 3.5)),
            'matched_dimu_mass': hist.Hist("Events", hist.Bin("mass", r"$mass_{J/\Psi} matched [GeV]$", 100, 2.95, 3.25)),
            'gen_dimu_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,J/\Psi} gen [GeV]$", 40, 0, 100)),
            'gen_dimu_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{J/\Psi} gen$", 60, -4, 4)),
            'gen_dimu_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{J/\Psi}$ gen [rad]", 60, -3.5, 3.5)),
            'gen_dimu_mass': hist.Hist("Events", hist.Bin("mass", r"$mass_{J/\Psi} gen [GeV]$", 100, 2.95, 3.25)),
            'all_gen_dimu_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,J/\Psi} all gen [GeV]", 40, 0, 80)),
            'all_gen_dimu_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{J/\Psi} all gen$", 60, -4, 4)),
            'all_gen_dimu_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{J/\Psi}$ all gen [rad]", 60, -3.5, 3.5)),
            'all_gen_dimu_mass': hist.Hist("Events", hist.Bin("mass", r"$mass_{J/\Psi} all gen$ [GeV]", 100, 2.95, 3.25)),
            'all_dimu_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,J/\Psi} all$ [GeV]", 40, 0, 80)),
            'all_dimu_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{J/\Psi} all$", 60, -4, 4)),
            'all_dimu_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{J/\Psi} all [rad]$ [rad]", 60, -3.5, 3.5)),
            'all_dimu_mass': hist.Hist("Events", hist.Bin("mass", r"$mass_{J/\Psi [GeV]} all [GeV]$", 100, 2.95, 3.25)),
            'matched_muon_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,\mu}$ matched [GeV]", 50, 0, 25)),
            'matched_muon_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{\mu} matched$", 60, -10, 10)),
            'matched_muon_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{\mu} matched [rad]$", 60, -3.5, 3.5)),
            'gen_muon_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,\mu} gen [GeV]$", 50, 0, 25)),
            'gen_muon_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{\mu} gen$", 60, -10, 10)),
            'gen_muon_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{\mu}$ gen [rad]", 60, -3.5, 3.5)),
            'all_gen_muon_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,\mu} all gen [GeV]", 50, 0, 25)),
            'all_gen_muon_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{\mu} all gen$", 60, -10, 10)),
            'all_gen_muon_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{\mu}$ all gen [rad]", 60, -3.5, 3.5)),
            'all_muon_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,\mu} all [GeV]$", 50, 0, 25)),
            'all_muon_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{\mu} all$", 60, -10, 10)),
            'all_muon_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{\mu} all [rad]$", 60, -3.5, 3.5)),
            'gen_muon_2d_pt': hist.Hist("Events", 
                                     hist.Bin("pt1", r"$p_{T, 1st \mu}$", 50, 0, 25), 
                                     hist.Bin("pt2", r"$p_{T, 2nd \mu}$", 50, 0, 25)),
            'gen_muon_2d_eta': hist.Hist("Events",
                                         hist.Bin("eta1", r"$\eta_{1st \mu}$", 60, -10, 10),
                                         hist.Bin("eta2", r"$\eta_{2nd \mu}$", 60, -10, 10)),
            'matched_dstar_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,D*} matched$", 40, 0, 20)),
            'matched_dstar_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{D*} matched$", 60, -4, 4)),
            'matched_dstar_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{D*}$ matched", 60, -3.5, 3.5)),
            'matched_dstar_deltamr': hist.Hist("Events", hist.Bin("deltamr", "$\Delta m_{D*}$ matched", 50, 0.138, 0.162)),
            'gen_dstar_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,D*} gen$", 40, 0, 20)),
            'gen_dstar_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{D*} gen$", 60, -4, 4)),
            'gen_dstar_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{D*}$ gen", 60, -3.5, 3.5)),
            'all_gen_dstar_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,D*} all gen$", 40, 0, 20)),
            'all_gen_dstar_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{D*} all gen$", 60, -4, 4)),
            'all_gen_dstar_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{D*}$ all gen", 60, -3.5, 3.5)),
            'all_dstar_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,D*} all$", 40, 0, 20)),
            'all_dstar_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{D*} all$", 60, -4, 4)),
            'all_dstar_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{D*}$ all", 60, -3.5, 3.5)),
            'all_dstar_deltamr': hist.Hist("Events", hist.Bin("deltamr", "$\Delta m_{D*}$ all", 50, 0.138, 0.162)),
            'all_dstar_D0cosphi' : hist.Hist("Events", hist.Bin("cosphi", r"$cos(\alpha)$", 800, -1, 1)),
            
            'gen_dstar_dimu' : hist.Hist("Events", 
                                   hist.Bin("dstar_pt", r"$p_{T, gen D*} [GeV]$", 45, 0, 15),
                                   hist.Bin("dimu_pt", r"$p_{T, gen J/\Psi}$ [GeV]", 60, 0, 30)),
            
            'reco_dstar_dimu' : hist.Hist("Events", 
                                   hist.Bin("dstar_pt", r"$p_{T, reco D*} [GeV]$", 45, 0, 15),
                                   hist.Bin("dimu_pt", r"$p_{T, reco J/\Psi}$ [GeV]", 60, 0, 30)),
            
            #################### Dstar-muon not finished ####################
            'gen_dstar_mu' : hist.Hist("Events", 
                                   hist.Bin("dstar_pt", r"$p_{T, gen D*} [GeV]$", 45, 0, 15),
                                   hist.Bin("muon_pt", r"$p_{T, gen \mu} [GeV]$", 50, 0, 25)),
            
            'reco_dstar_mu' : hist.Hist("Events", 
                                   hist.Bin("dstar_pt", r"$p_{T, reco D*} [GeV]$", 45, 0, 15),
                                   hist.Bin("muon_pt", r"$p_{T, reco \mu} [GeV]$", 50, 0, 25)),
            
            #################### NEW!! Associated ####################
            
            ## Associated Dstar
            'asso_dstar_deltamr': hist.Hist("Events", hist.Bin("deltamr", "$\Delta m_{D*}$ asso", 25, 0.138, 0.162)),
            'asso_dstar_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,D*} asso$", 20, 0, 20)),
            'asso_dstar_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{D*} asso$", 30, -4, 4)),
            'asso_dstar_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{D*}$ asso", 30, -3.5, 3.5)),
            
            'Dstar_D0cosphi' : hist.Hist("Events", hist.Bin("cosphi", r"$cos(\alpha)$", 100, -1, 1)),
            
            ## Associated Jpsi
            'asso_dimu_mass': hist.Hist("Events", hist.Bin("mass", r"$mass_{J/\Psi} asso [GeV]$", 50, 2.95, 3.25)),
            'asso_dimu_pt': hist.Hist("Events", hist.Bin("pt", r"$p_{T,J/\Psi} asso$ [GeV]", 20, 0, 80)),
            'asso_dimu_eta': hist.Hist("Events", hist.Bin("eta", r"$\eta_{J/\Psi} asso$", 30, -4, 4)),
            'asso_dimu_phi': hist.Hist("Events", hist.Bin("phi", r"$\phi_{J/\Psi} asso$ [rad]", 30, -3.5, 3.5)),
            
            ## Associated JpsiDstar
            'JpsiDstar_deltarap': hist.Hist("Events", hist.Bin("deltarap", "$\Delta y$", 25, -5, 5)),
            'JpsiDstar_mass': hist.Hist("Events", hist.Bin("mass", "$m_{J/\psi D*}$ [GeV]", 25, 0, 100)),
            
            
            
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        output = self.accumulator.identity()
        
        # test if there is any events in the file
        if len(events) == 0:
            return output
        
        # Collection extraction
        Dimus = ak.zip({**get_vars_dict(events, dimu_cols)}, with_name="PtEtaPhiMCandidate")
        Muons = ak.zip({**get_vars_dict(events, muon_cols)}, with_name="PtEtaPhiMCandidate")
        Dstars = ak.zip({'mass': (events.Dstar_D0mass + events.Dstar_deltamr),
                        'charge': events.Dstar_pischg,
                        **get_vars_dict(events, dstar_cols)}, 
                        with_name="PtEtaPhiMCandidate")
        
        output['cutflow']['Number of events'] += len(events)
        output['cutflow']['Number of initial Dimuons'] += ak.sum(ak.num(Dimus))
        output['cutflow']['Number of initial Muons'] += ak.sum(ak.num(Muons))
        
        # Triggers for 2017 charmonium
        try:
            hlt_char_2017 = ak.zip({**get_vars_dict(events, hlt_cols_charm_2017)})
        except:
            hlt_char_2017 = ak.zip({"HLT_2017" : "HLT_2017"})
        
        ##### Trigger cut

        # Activate trigger
        hlt = False
        # HLT to be used
        hlt_filter = 'HLT_Dimuon0_Jpsi'

        # Trigger choice
        if hlt:
            trigger_cut = hlt_char_2017[hlt_filter]
            hlt_char_2017 = hlt_char_2017[hlt_filter]
                
        if not hlt:
            #print("You are not running with trigger")
            # Assign 1 to all events.
            trigger_cut = np.ones(len(Dimus), dtype=bool)
            
        # Trigger filter
        Dimu = Dimus[trigger_cut]
        Muon = Muons[trigger_cut]
        Dstar = Dstars[trigger_cut]
        
        ## Rec dimuon cuts
        
        # Dimu charge = 0 cut
        if 'like_charge_dimuon' in cut_list:
            Dimu = Dimus[Dimu.charge == 0]
        output['cutflow']['Dimu 0 charge'] += ak.sum(ak.num(Dimu))
        
        # Jpsi mass range cut
        if 'jpsi_mass' in cut_list:
            Dimu = Dimu[(Dimu.mass > 2.95) & (Dimu.mass < 3.25)]
        output['cutflow']['Jpsi mass'] += ak.sum(ak.num(Dimu))

        # Get the Muons from Dimu, for cuts in their params
        Muon = ak.zip({'0': Muons[Dimu.t1muIdx], '1': Muons[Dimu.t2muIdx]})
        
        ## Rec muon cuts
        
        # Muon softId cuts
        if 'soft_id' in cut_list:
            soft_id = (Muon.slot0.softId > 0) & (Muon.slot1.softId > 0)
            Dimu = Dimu[soft_id]
            Muon = Muon[soft_id]
        output['cutflow']['Dimu muon softId'] += ak.sum(ak.num(Dimu))

        # Muon pt cut
        if 'muon_pt' in cut_list:
            muon_pt_cut = (Muon.slot0.pt > 3) & (Muon.slot1.pt > 3)
            Dimu = Dimu[muon_pt_cut]
            Muon = Muon[muon_pt_cut]
        output['cutflow']['Dimu muon pt cut'] += ak.sum(ak.num(Dimu))
        
        # Muon eta cut
        if 'muon_eta' in cut_list:
            muon_eta_cut = (np.absolute(Muon.slot0.eta) <= 2.4) & (np.absolute(Muon.slot1.eta) <= 2.4)
            Dimu = Dimu[muon_eta_cut]
            Muon = Muon[muon_eta_cut]  
        output['cutflow']['Dimu before gen match'] += ak.sum(ak.num(Dimu))
        
        # Number of events with muons.
        evts_with_muon = Muon[ak.num(Muon) > 0]
        output['cutflow']['Number of evts with Muon'] += len(evts_with_muon)
    
        ## Cuts for Dstar
        
        # Dstar has not muon cut
        if 'dstar_hasmuon' in cut_list:
            Dstar = Dstars[~Dstars.hasMuon]
        output['cutflow']['Dstar trk muon cut'] += ak.sum(ak.num(Dstar))
        
        # Dstar trk pt cut
        if 'dstar_trk_pt' in cut_list:
            Dstar = Dstar[(Dstar.Kpt > 0.5) & (Dstar.pipt > 0.5)]
        output['cutflow']['Dstar trk pt cut'] += ak.sum(ak.num(Dstar))

        # Dstar trk chi2 cut
        if 'dstar_trk_chindof' in cut_list:
            Dstar = Dstar[(Dstar.Kchindof < 2.5) & (Dstar.pichindof < 2.5)]
        output['cutflow']['Dstar chindof pt cut'] += ak.sum(ak.num(Dstar))

        # Dstar trk hits cut
        if 'dstar_trk_hits' in cut_list:
            Dstar = Dstar[(Dstar.KnValid > 4) & (Dstar.pinValid > 4) & (Dstar.KnPix > 1) & (Dstar.pinPix > 1)]
        output['cutflow']['Dstar trk hits cut'] += ak.sum(ak.num(Dstar))

        # Dstar trk impact xy cut
        if 'dstar_trk_impact_xy' in cut_list:
            Dstar = Dstar[(Dstar.Kdxy < 0.1) & (Dstar.pidxy < 0.1)]
        output['cutflow']['Dstar trk impact xy cut'] += ak.sum(ak.num(Dstar))

        # Dstar trk impact z cut
        if 'dstar_trk_impact_z' in cut_list:
            Dstar = Dstar[(Dstar.Kdz < 1) & (Dstar.pidz < 1)]
        output['cutflow']['Dstar trk impact z cut'] += ak.sum(ak.num(Dstar))

        # Dstar pis pt
        if 'dstar_pis_pt' in cut_list:
            Dstar = Dstar[Dstar.pispt > 0.3]
        output['cutflow']['Dstar pis pt cut'] += ak.sum(ak.num(Dstar))

        # Dstar pis chindof
        if 'dstar_pis_chindof' in cut_list:
            Dstar = Dstar[Dstar.pischindof < 3]
        output['cutflow']['Dstar pis chi2 cut'] += ak.sum(ak.num(Dstar))

        # Dstar pis nvalid
        if 'dstar_pis_nvalid' in cut_list:
            Dstar = Dstar[Dstar.pisnValid > 2]
        output['cutflow']['Dstar pis hits cut'] += ak.sum(ak.num(Dstar))

        # D0 of Dstar cosphi
        if 'dstar_d0_cosphi' in cut_list:
            Dstar = Dstar[Dstar.D0cosphi > 0.99]
        output['cutflow']['Dstar D0 cosphi cut'] += ak.sum(ak.num(Dstar))

        # D0 of Dstar mass
        if 'dstar_d0_mass' in cut_list:
            Dstar = Dstar[(Dstar.D0mass < D0_PDG_MASS + 0.028) & (Dstar.D0mass > D0_PDG_MASS - 0.028)]
        output['cutflow']['Dstar D0 mass cut'] += ak.sum(ak.num(Dstar))

        if 'dstar_d0_pt' in cut_list:
            Dstar = Dstar[Dstar.D0pt > 4]
        output['cutflow']['Dstar D0 pt cut'] += ak.sum(ak.num(Dstar))
    
        if 'dstar_d0_dlsig' in cut_list:
            Dstar = Dstar[Dstar.D0dlSig > 3]
        output['cutflow']['Dstar D0 dlSig cut'] += ak.sum(ak.num(Dstar))

        Dstar['wrg_chg'] = (Dstar.Kchg == Dstar.pichg)
        
        if 'dstar_pt' in cut_list:
            Dstar = Dstar[Dstar.pt > 4]
        output['cutflow']['Dstar pt cut'] += ak.sum(ak.num(Dstar))
        
        if 'dstar_right_charge' in cut_list:
            Dstar = Dstar[~Dstar.wrg_chg]
        output['cutflow']['Dstar right charge cut'] += ak.sum(ak.num(Dstar))
        
        ## Dimu + Dstar
        
        asso = ak.cartesian([Dimu, Dstar]) 
        if 'association' in cut_list:
            asso = asso[asso.slot0.vtxIdx == asso.slot1.vtxIdx]
                
        Dimu_asso = ak.zip({
                   'pt': asso.slot0.pt,
                   'eta': asso.slot0.eta,
                   'phi': asso.slot0.phi,
                   'mass': asso.slot0.mass,
                   'charge': asso.slot0.charge}, with_name="PtEtaPhiMCandidate")
       
        Dstar_asso = ak.zip({
                    'pt': asso.slot1.pt,
                    'eta': asso.slot1.eta,
                    'phi': asso.slot1.phi,
                    'mass': asso.slot1.deltamr,
                    'charge': asso.slot1.charge}, with_name="PtEtaPhiMCandidate")
        
        asso['deltarap'] = asso.slot0.rap - asso.slot1.rap
        asso['cand'] = Dimu_asso + Dstar_asso
        
        DimuDstar = asso
        
        DimuDstar_p4 = ak.zip({'x': DimuDstar.cand.x, 
                               'y': DimuDstar.cand.y,
                               'z': DimuDstar.cand.z,
                               't': DimuDstar.cand.t}, with_name="LorentzVector")
        output['cutflow']['Number of DimuDstar'] += ak.sum(ak.num(DimuDstar))
        output['cutflow']['Number of Associated Dstar'] += ak.sum(ak.num(Dstar_asso.mass))
        
        evts_with_dimudstar  = DimuDstar[ak.num(DimuDstar) > 0]
        output['cutflow']['Number of evts with DimuDstar'] += len(evts_with_dimudstar)
        
        # Dstar
        output['asso_dstar_deltamr'].fill(deltamr = ak.flatten(Dstar_asso.mass))
        output['asso_dstar_pt'].fill(pt = ak.flatten(Dstar_asso.pt))
        output['asso_dstar_eta'].fill(eta = ak.flatten(Dstar_asso.eta))
        output['asso_dstar_phi'].fill(phi = ak.flatten(Dstar_asso.phi))
        
        # Jpsi
        output['asso_dimu_mass'].fill(mass = ak.flatten(Dimu_asso.mass))
        output['asso_dimu_pt'].fill(pt = ak.flatten(Dimu_asso.pt))
        output['asso_dimu_eta'].fill(eta = ak.flatten(Dimu_asso.eta))
        output['asso_dimu_phi'].fill(phi = ak.flatten(Dimu_asso.phi))
        
        # JpsiDstar
        output['JpsiDstar_mass'].fill(mass = ak.flatten(DimuDstar_p4.mass))
        output['JpsiDstar_deltarap'].fill(deltarap = ak.flatten(DimuDstar.deltarap))
        
        # Retrieves the associated dimuon
        reco_dimu_plot = DimuDstar.slot0.slot0
        # Retrieves the associated dstar
        reco_dstar_plot = DimuDstar.slot1.slot0
        
        # Fills the histogram
        output['reco_dstar_dimu'].fill(dstar_pt=ak.flatten(Dstar_asso.pt),
                                       dimu_pt=ak.flatten(Dimu_asso.pt),)
        
        return output

    def postprocess(self, accumulator):
        return accumulator

#  L0 - No cuts

In [24]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = []

output_l0 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l0['cutflow'])

nevt_l0 = output_l0['cutflow']['Number of events']
JpsiDstar_l0 = output_l0['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l0 = output_l0['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L0")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 472.15 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 6865, 'Jpsi mass': 6865, 'Dimu muon softId': 6865, 'Dimu muon pt cut': 6865, 'Dimu before gen match': 6865, 'Number of evts with Muon': 4625, 'Dstar trk muon cut': 1002236, 'Dstar trk pt cut': 1002236, 'Dstar chindof pt cut': 1002236, 'Dstar trk hits cut': 1002236, 'Dstar trk impact xy cut': 1002236, 'Dstar trk impact z cut': 1002236, 'Dstar pis pt cut': 1002236, 'Dstar pis chi2 cut': 1002236, 'Dstar pis hits cut': 1002236, 'Dstar D0 cosphi cut': 1002236, 'Dstar D0 mass cut': 1002236, 'Dstar D0 pt cut': 1002236, 'Dstar D0 dlSig cut': 1002236, 'Dstar pt cut': 1002236, 'Dstar right charge cut': 1002236, 'Number of DimuDstar': 1304281, 'Number of Associated Dstar': 1304281, 'Number of evts with DimuDstar': 4617})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake

#  L1

In [25]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon']

output_l1 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l1['cutflow'])

nevt_l1 = output_l1['cutflow']['Number of events']
JpsiDstar_l1 = output_l1['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l1 = output_l1['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L1")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 326.70 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 5534, 'Dimu muon softId': 5534, 'Dimu muon pt cut': 5534, 'Dimu before gen match': 5534, 'Number of evts with Muon': 4428, 'Dstar trk muon cut': 1002236, 'Dstar trk pt cut': 1002236, 'Dstar chindof pt cut': 1002236, 'Dstar trk hits cut': 1002236, 'Dstar trk impact xy cut': 1002236, 'Dstar trk impact z cut': 1002236, 'Dstar pis pt cut': 1002236, 'Dstar pis chi2 cut': 1002236, 'Dstar pis hits cut': 1002236, 'Dstar D0 cosphi cut': 1002236, 'Dstar D0 mass cut': 1002236, 'Dstar D0 pt cut': 1002236, 'Dstar D0 dlSig cut': 1002236, 'Dstar pt cut': 1002236, 'Dstar right charge cut': 1002236, 'Number of DimuDstar': 996743, 'Number of Associated Dstar': 996743, 'Number of evts with DimuDstar': 4420})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake',

#  L2 - 

In [26]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass']

output_l2 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l2['cutflow'])

nevt_l2 = output_l2['cutflow']['Number of events']
JpsiDstar_l2 = output_l2['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l2 = output_l2['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L2")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 269.40 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4371, 'Dimu muon pt cut': 4371, 'Dimu before gen match': 4371, 'Number of evts with Muon': 4156, 'Dstar trk muon cut': 1002236, 'Dstar trk pt cut': 1002236, 'Dstar chindof pt cut': 1002236, 'Dstar trk hits cut': 1002236, 'Dstar trk impact xy cut': 1002236, 'Dstar trk impact z cut': 1002236, 'Dstar pis pt cut': 1002236, 'Dstar pis chi2 cut': 1002236, 'Dstar pis hits cut': 1002236, 'Dstar D0 cosphi cut': 1002236, 'Dstar D0 mass cut': 1002236, 'Dstar D0 pt cut': 1002236, 'Dstar D0 dlSig cut': 1002236, 'Dstar pt cut': 1002236, 'Dstar right charge cut': 1002236, 'Number of DimuDstar': 734046, 'Number of Associated Dstar': 734046, 'Number of evts with DimuDstar': 4148})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake',

#  L3 - 

In [27]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id']

output_l3 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l3['cutflow'])

nevt_l3 = output_l3['cutflow']['Number of events']
JpsiDstar_l3 = output_l3['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l3 = output_l3['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L3")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 268.45 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 4005, 'Dimu before gen match': 4005, 'Number of evts with Muon': 3965, 'Dstar trk muon cut': 1002236, 'Dstar trk pt cut': 1002236, 'Dstar chindof pt cut': 1002236, 'Dstar trk hits cut': 1002236, 'Dstar trk impact xy cut': 1002236, 'Dstar trk impact z cut': 1002236, 'Dstar pis pt cut': 1002236, 'Dstar pis chi2 cut': 1002236, 'Dstar pis hits cut': 1002236, 'Dstar D0 cosphi cut': 1002236, 'Dstar D0 mass cut': 1002236, 'Dstar D0 pt cut': 1002236, 'Dstar D0 dlSig cut': 1002236, 'Dstar pt cut': 1002236, 'Dstar right charge cut': 1002236, 'Number of DimuDstar': 651210, 'Number of Associated Dstar': 651210, 'Number of evts with DimuDstar': 3957})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake',

#  L4 - 

In [28]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt']

output_l4 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l4['cutflow'])

nevt_l4 = output_l4['cutflow']['Number of events']
JpsiDstar_l4 = output_l4['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l4 = output_l4['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L4")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 269.86 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3250, 'Number of evts with Muon': 3249, 'Dstar trk muon cut': 1002236, 'Dstar trk pt cut': 1002236, 'Dstar chindof pt cut': 1002236, 'Dstar trk hits cut': 1002236, 'Dstar trk impact xy cut': 1002236, 'Dstar trk impact z cut': 1002236, 'Dstar pis pt cut': 1002236, 'Dstar pis chi2 cut': 1002236, 'Dstar pis hits cut': 1002236, 'Dstar D0 cosphi cut': 1002236, 'Dstar D0 mass cut': 1002236, 'Dstar D0 pt cut': 1002236, 'Dstar D0 dlSig cut': 1002236, 'Dstar pt cut': 1002236, 'Dstar right charge cut': 1002236, 'Number of DimuDstar': 525275, 'Number of Associated Dstar': 525275, 'Number of evts with DimuDstar': 3242})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake',

#  L5 - 

In [29]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta']

output_l5 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l5['cutflow'])

nevt_l5 = output_l5['cutflow']['Number of events']
JpsiDstar_l5 = output_l5['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l5 = output_l5['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L5")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 273.72 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 1002236, 'Dstar trk pt cut': 1002236, 'Dstar chindof pt cut': 1002236, 'Dstar trk hits cut': 1002236, 'Dstar trk impact xy cut': 1002236, 'Dstar trk impact z cut': 1002236, 'Dstar pis pt cut': 1002236, 'Dstar pis chi2 cut': 1002236, 'Dstar pis hits cut': 1002236, 'Dstar D0 cosphi cut': 1002236, 'Dstar D0 mass cut': 1002236, 'Dstar D0 pt cut': 1002236, 'Dstar D0 dlSig cut': 1002236, 'Dstar pt cut': 1002236, 'Dstar right charge cut': 1002236, 'Number of DimuDstar': 519762, 'Number of Associated Dstar': 519762, 'Number of evts with DimuDstar': 3204})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake',

#  L6 - 

In [30]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon']

output_l6 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l6['cutflow'])

nevt_l6 = output_l6['cutflow']['Number of events']
JpsiDstar_l6 = output_l6['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l6 = output_l6['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L6")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 268.38 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 974716, 'Dstar chindof pt cut': 974716, 'Dstar trk hits cut': 974716, 'Dstar trk impact xy cut': 974716, 'Dstar trk impact z cut': 974716, 'Dstar pis pt cut': 974716, 'Dstar pis chi2 cut': 974716, 'Dstar pis hits cut': 974716, 'Dstar D0 cosphi cut': 974716, 'Dstar D0 mass cut': 974716, 'Dstar D0 pt cut': 974716, 'Dstar D0 dlSig cut': 974716, 'Dstar pt cut': 974716, 'Dstar right charge cut': 974716, 'Number of DimuDstar': 505464, 'Number of Associated Dstar': 505464, 'Number of evts with DimuDstar': 3204})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'S

#  L7 - 

In [31]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt']

output_l7 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l7['cutflow'])

nevt_l7 = output_l7['cutflow']['Number of events']
JpsiDstar_l7 = output_l7['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l7 = output_l7['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L7")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 270.45 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 922948, 'Dstar trk hits cut': 922948, 'Dstar trk impact xy cut': 922948, 'Dstar trk impact z cut': 922948, 'Dstar pis pt cut': 922948, 'Dstar pis chi2 cut': 922948, 'Dstar pis hits cut': 922948, 'Dstar D0 cosphi cut': 922948, 'Dstar D0 mass cut': 922948, 'Dstar D0 pt cut': 922948, 'Dstar D0 dlSig cut': 922948, 'Dstar pt cut': 922948, 'Dstar right charge cut': 922948, 'Number of DimuDstar': 478670, 'Number of Associated Dstar': 478670, 'Number of evts with DimuDstar': 3203})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'S

#  L8 - 

In [32]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof']

output_l8 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l8['cutflow'])

nevt_l8 = output_l8['cutflow']['Number of events']
JpsiDstar_l8 = output_l8['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l8 = output_l8['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L8")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 325.42 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 825053, 'Dstar trk impact xy cut': 825053, 'Dstar trk impact z cut': 825053, 'Dstar pis pt cut': 825053, 'Dstar pis chi2 cut': 825053, 'Dstar pis hits cut': 825053, 'Dstar D0 cosphi cut': 825053, 'Dstar D0 mass cut': 825053, 'Dstar D0 pt cut': 825053, 'Dstar D0 dlSig cut': 825053, 'Dstar pt cut': 825053, 'Dstar right charge cut': 825053, 'Number of DimuDstar': 427543, 'Number of Associated Dstar': 427543, 'Number of evts with DimuDstar': 3202})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'S

#  L9 - 

In [33]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits']

output_l9 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l9['cutflow'])

nevt_l9 = output_l9['cutflow']['Number of events']
JpsiDstar_l9 = output_l9['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l9 = output_l9['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L9")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 344.75 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 646389, 'Dstar trk impact z cut': 646389, 'Dstar pis pt cut': 646389, 'Dstar pis chi2 cut': 646389, 'Dstar pis hits cut': 646389, 'Dstar D0 cosphi cut': 646389, 'Dstar D0 mass cut': 646389, 'Dstar D0 pt cut': 646389, 'Dstar D0 dlSig cut': 646389, 'Dstar pt cut': 646389, 'Dstar right charge cut': 646389, 'Number of DimuDstar': 335567, 'Number of Associated Dstar': 335567, 'Number of evts with DimuDstar': 3197})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'S

#  L10 - 

In [34]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy']

output_l10 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l10['cutflow'])

nevt_l10 = output_l10['cutflow']['Number of events']
JpsiDstar_l10 = output_l10['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l10 = output_l10['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L10")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 349.95 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639026, 'Dstar pis pt cut': 639026, 'Dstar pis chi2 cut': 639026, 'Dstar pis hits cut': 639026, 'Dstar D0 cosphi cut': 639026, 'Dstar D0 mass cut': 639026, 'Dstar D0 pt cut': 639026, 'Dstar D0 dlSig cut': 639026, 'Dstar pt cut': 639026, 'Dstar right charge cut': 639026, 'Number of DimuDstar': 331748, 'Number of Associated Dstar': 331748, 'Number of evts with DimuDstar': 3196})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'S

#  L11 - 

In [35]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z']

output_l11 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l11['cutflow'])

nevt_l11 = output_l11['cutflow']['Number of events']
JpsiDstar_l11 = output_l11['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l11 = output_l11['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L11")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 349.09 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 639009, 'Dstar pis chi2 cut': 639009, 'Dstar pis hits cut': 639009, 'Dstar D0 cosphi cut': 639009, 'Dstar D0 mass cut': 639009, 'Dstar D0 pt cut': 639009, 'Dstar D0 dlSig cut': 639009, 'Dstar pt cut': 639009, 'Dstar right charge cut': 639009, 'Number of DimuDstar': 331739, 'Number of Associated Dstar': 331739, 'Number of evts with DimuDstar': 3196})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'S

#  L12 - 

In [36]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt']

output_l12 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l12['cutflow'])

nevt_l12 = output_l12['cutflow']['Number of events']
JpsiDstar_l12 = output_l12['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l12 = output_l12['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L12")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 346.63 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 74535, 'Dstar pis hits cut': 74535, 'Dstar D0 cosphi cut': 74535, 'Dstar D0 mass cut': 74535, 'Dstar D0 pt cut': 74535, 'Dstar D0 dlSig cut': 74535, 'Dstar pt cut': 74535, 'Dstar right charge cut': 74535, 'Number of DimuDstar': 38841, 'Number of Associated Dstar': 38841, 'Number of evts with DimuDstar': 2986})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysi

#  L13 - 

In [37]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt', 'dstar_pis_chindof']

output_l13 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l13['cutflow'])

nevt_l13 = output_l13['cutflow']['Number of events']
JpsiDstar_l13 = output_l13['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l13 = output_l13['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L13")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 329.57 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 71897, 'Dstar pis hits cut': 71897, 'Dstar D0 cosphi cut': 71897, 'Dstar D0 mass cut': 71897, 'Dstar D0 pt cut': 71897, 'Dstar D0 dlSig cut': 71897, 'Dstar pt cut': 71897, 'Dstar right charge cut': 71897, 'Number of DimuDstar': 37495, 'Number of Associated Dstar': 37495, 'Number of evts with DimuDstar': 2972})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysi

#  L14 - 

In [38]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt', 'dstar_pis_chindof', 'dstar_pis_nvalid']

output_l14 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l14['cutflow'])

nevt_l14 = output_l14['cutflow']['Number of events']
JpsiDstar_l14 = output_l14['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l14 = output_l14['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L14")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 330.01 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 71897, 'Dstar pis hits cut': 71897, 'Dstar D0 cosphi cut': 71897, 'Dstar D0 mass cut': 71897, 'Dstar D0 pt cut': 71897, 'Dstar D0 dlSig cut': 71897, 'Dstar pt cut': 71897, 'Dstar right charge cut': 71897, 'Number of DimuDstar': 37495, 'Number of Associated Dstar': 37495, 'Number of evts with DimuDstar': 2972})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysi

#  L15 - 

In [39]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt', 'dstar_pis_chindof', 'dstar_pis_nvalid',
            'dstar_d0_cosphi']

output_l15 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l15['cutflow'])

nevt_l15 = output_l15['cutflow']['Number of events']
JpsiDstar_l15 = output_l15['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l15 = output_l15['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L15")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 320.70 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 71897, 'Dstar pis hits cut': 71897, 'Dstar D0 cosphi cut': 7518, 'Dstar D0 mass cut': 7518, 'Dstar D0 pt cut': 7518, 'Dstar D0 dlSig cut': 7518, 'Dstar pt cut': 7518, 'Dstar right charge cut': 7518, 'Number of DimuDstar': 3961, 'Number of Associated Dstar': 3961, 'Number of evts with DimuDstar': 1699})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysisbot', '

#  L16 - 

In [40]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt', 'dstar_pis_chindof', 'dstar_pis_nvalid',
            'dstar_d0_cosphi', 'dstar_d0_mass']

output_l16 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l16['cutflow'])

nevt_l16 = output_l16['cutflow']['Number of events']
JpsiDstar_l16 = output_l16['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l16 = output_l16['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L16")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 303.49 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 71897, 'Dstar pis hits cut': 71897, 'Dstar D0 cosphi cut': 7518, 'Dstar D0 mass cut': 3635, 'Dstar D0 pt cut': 3635, 'Dstar D0 dlSig cut': 3635, 'Dstar pt cut': 3635, 'Dstar right charge cut': 3635, 'Number of DimuDstar': 1952, 'Number of Associated Dstar': 1952, 'Number of evts with DimuDstar': 1154})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysisbot', '

#  L17 - 

In [41]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt', 'dstar_pis_chindof', 'dstar_pis_nvalid',
            'dstar_d0_cosphi', 'dstar_d0_mass', 'dstar_d0_pt']

output_l17 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l17['cutflow'])

nevt_l17 = output_l17['cutflow']['Number of events']
JpsiDstar_l17 = output_l17['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l17 = output_l17['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L17")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 300.69 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 71897, 'Dstar pis hits cut': 71897, 'Dstar D0 cosphi cut': 7518, 'Dstar D0 mass cut': 3635, 'Dstar D0 pt cut': 1579, 'Dstar D0 dlSig cut': 1579, 'Dstar pt cut': 1579, 'Dstar right charge cut': 1579, 'Number of DimuDstar': 835, 'Number of Associated Dstar': 835, 'Number of evts with DimuDstar': 581})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysisbot', 'can

#  L18 - 

In [42]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt', 'dstar_pis_chindof', 'dstar_pis_nvalid',
            'dstar_d0_cosphi', 'dstar_d0_mass', 'dstar_d0_pt', 'dstar_d0_dlsig']

output_l18 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l18['cutflow'])

nevt_l18 = output_l18['cutflow']['Number of events']
JpsiDstar_l18 = output_l18['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l18 = output_l18['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L18")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 302.09 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 71897, 'Dstar pis hits cut': 71897, 'Dstar D0 cosphi cut': 7518, 'Dstar D0 mass cut': 3635, 'Dstar D0 pt cut': 1579, 'Dstar D0 dlSig cut': 551, 'Dstar pt cut': 551, 'Dstar right charge cut': 551, 'Number of DimuDstar': 310, 'Number of Associated Dstar': 310, 'Number of evts with DimuDstar': 240})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysisbot', 'can_re

#  L19 - 

In [43]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt', 'dstar_pis_chindof', 'dstar_pis_nvalid',
            'dstar_d0_cosphi', 'dstar_d0_mass', 'dstar_d0_pt', 'dstar_d0_dlsig',
            'dstar_pt']

output_l19 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l19['cutflow'])

nevt_l19 = output_l19['cutflow']['Number of events']
JpsiDstar_l19 = output_l19['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l19 = output_l19['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L19")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 302.72 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 71897, 'Dstar pis hits cut': 71897, 'Dstar D0 cosphi cut': 7518, 'Dstar D0 mass cut': 3635, 'Dstar D0 pt cut': 1579, 'Dstar D0 dlSig cut': 551, 'Dstar pt cut': 551, 'Dstar right charge cut': 551, 'Number of DimuDstar': 310, 'Number of Associated Dstar': 310, 'Number of evts with DimuDstar': 240})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysisbot', 'can_re

#  L20 - 

In [44]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt', 'dstar_pis_chindof', 'dstar_pis_nvalid',
            'dstar_d0_cosphi', 'dstar_d0_mass', 'dstar_d0_pt', 'dstar_d0_dlsig',
            'dstar_pt', 'dstar_right_charge']

output_l20 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l20['cutflow'])

nevt_l20 = output_l20['cutflow']['Number of events']
JpsiDstar_l20 = output_l20['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l20 = output_l20['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L20")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 304.27 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 71897, 'Dstar pis hits cut': 71897, 'Dstar D0 cosphi cut': 7518, 'Dstar D0 mass cut': 3635, 'Dstar D0 pt cut': 1579, 'Dstar D0 dlSig cut': 551, 'Dstar pt cut': 551, 'Dstar right charge cut': 473, 'Number of DimuDstar': 275, 'Number of Associated Dstar': 275, 'Number of evts with DimuDstar': 217})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysisbot', 'can_re

#  L21 - All cuts

In [45]:
data = {"test": files[0:600]}
                      
tstart = time.time()

cut_list = ['like_charge_dimuon', 'jpsi_mass', 'soft_id', 'muon_pt', 'muon_eta', 'dstar_hasmuon',
            'dstar_trk_pt', 'dstar_trk_chindof', 'dstar_trk_hits', 'dstar_trk_impact_xy',
            'dstar_trk_impact_z', 'dstar_pis_pt', 'dstar_pis_chindof', 'dstar_pis_nvalid',
            'dstar_d0_cosphi', 'dstar_d0_mass', 'dstar_d0_pt', 'dstar_d0_dlsig',
            'dstar_pt', 'dstar_right_charge', 'association']

output_l21 = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=GenTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")
print(output_l21['cutflow'])

nevt_l21 = output_l21['cutflow']['Number of events']
JpsiDstar_l21 = output_l21['cutflow']['Number of DimuDstar']
nevts_with_jpsi_dstar_l21 = output_l21['cutflow']['Number of evts with DimuDstar']

bot.bot_message("Process Finished: L21")

Processing:   0%|          | 0/600 [00:00<?, ?chunk/s]

Process finished in: 297.15 s
defaultdict_accumulator(<class 'int'>, {'Number of events': 6033, 'Number of initial Dimuons': 6865, 'Number of initial Muons': 36118, 'Dimu 0 charge': 5534, 'Jpsi mass': 4371, 'Dimu muon softId': 4005, 'Dimu muon pt cut': 3250, 'Dimu before gen match': 3212, 'Number of evts with Muon': 3211, 'Dstar trk muon cut': 974716, 'Dstar trk pt cut': 922948, 'Dstar chindof pt cut': 825053, 'Dstar trk hits cut': 646389, 'Dstar trk impact xy cut': 639026, 'Dstar trk impact z cut': 639009, 'Dstar pis pt cut': 74535, 'Dstar pis chi2 cut': 71897, 'Dstar pis hits cut': 71897, 'Dstar D0 cosphi cut': 7518, 'Dstar D0 mass cut': 3635, 'Dstar D0 pt cut': 1579, 'Dstar D0 dlSig cut': 551, 'Dstar pt cut': 551, 'Dstar right charge cut': 473, 'Number of DimuDstar': 194, 'Number of Associated Dstar': 194, 'Number of evts with DimuDstar': 168})
Bot informations: {'id': 5179895518, 'can_join_groups': True, 'is_bot': True, 'first_name': 'Snake', 'username': 'SnakeAnalysisbot', 'can_re

In [79]:
with open('cutflow.csv', 'w') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    header = ['Cut Number', 'Cut applied', 'N Candidates', 'N events with  J/Ψ + D*', 'Drop w.r.t previous', 'Drop w.r.t first']
    writer.writerow(header)
    
    l0 = [0, 'No cut', JpsiDstar_l0, nevts_with_jpsi_dstar_l0, '-', '-']
    writer.writerow(l0)
    
    jpsi_cuts = ['J/Ψ']
    writer.writerow(jpsi_cuts)
    
    l1 = [1, 'Dimuon charge = 0', JpsiDstar_l1, nevts_with_jpsi_dstar_l1, nevts_with_jpsi_dstar_l1/nevts_with_jpsi_dstar_l0, nevts_with_jpsi_dstar_l1/nevts_with_jpsi_dstar_l0]
    writer.writerow(l1)
    
    l2 = [2, '2.95 < M(𝜇𝜇) < 3.25 GeV', JpsiDstar_l2, nevts_with_jpsi_dstar_l2, nevts_with_jpsi_dstar_l2/nevts_with_jpsi_dstar_l1, nevts_with_jpsi_dstar_l2/nevts_with_jpsi_dstar_l0]
    writer.writerow(l2)
    
    l3 = [3, 'Muon Soft_id', JpsiDstar_l3, nevts_with_jpsi_dstar_l3, nevts_with_jpsi_dstar_l3/nevts_with_jpsi_dstar_l2, nevts_with_jpsi_dstar_l3/nevts_with_jpsi_dstar_l0]
    writer.writerow(l3)
    
    l4 = [4, 'pT𝜇 > 3 GeV', JpsiDstar_l4, nevts_with_jpsi_dstar_l4, nevts_with_jpsi_dstar_l4/nevts_with_jpsi_dstar_l3, nevts_with_jpsi_dstar_l4/nevts_with_jpsi_dstar_l0]
    writer.writerow(l4)
    
    l5 = [5, '|𝜂𝜇| < 2.4', JpsiDstar_l5, nevts_with_jpsi_dstar_l5, nevts_with_jpsi_dstar_l5/nevts_with_jpsi_dstar_l4, nevts_with_jpsi_dstar_l5/nevts_with_jpsi_dstar_l0]
    writer.writerow(l5)
    
    dstar_cuts = ['D*']
    writer.writerow(dstar_cuts)
    
    l6 = [6, 'D* has no muon', JpsiDstar_l6, nevts_with_jpsi_dstar_l6, nevts_with_jpsi_dstar_l6/nevts_with_jpsi_dstar_l5, nevts_with_jpsi_dstar_l6/nevts_with_jpsi_dstar_l0]
    writer.writerow(l6)
    
    l7 = [7, 'Kaon and pion from D* pT > 0.5 GeV', JpsiDstar_l7, nevts_with_jpsi_dstar_l7, nevts_with_jpsi_dstar_l7/nevts_with_jpsi_dstar_l6, nevts_with_jpsi_dstar_l7/nevts_with_jpsi_dstar_l0]
    writer.writerow(l7)
    
    l8 = [8, 'Kaon and pion track 𝜒²/ndof < 2.5', JpsiDstar_l8, nevts_with_jpsi_dstar_l8, nevts_with_jpsi_dstar_l8/nevts_with_jpsi_dstar_l7, nevts_with_jpsi_dstar_l8/nevts_with_jpsi_dstar_l0]
    writer.writerow(l8)
    
    l9 = [9, 'Kaon and pion hits: valid hits > 4, pixel hits > 1', JpsiDstar_l9, nevts_with_jpsi_dstar_l9, nevts_with_jpsi_dstar_l9/nevts_with_jpsi_dstar_l8, nevts_with_jpsi_dstar_l9/nevts_with_jpsi_dstar_l0]
    writer.writerow(l9)
    
    l10 = [10, 'Kaon and Pion dxy < 0.1 cm', JpsiDstar_l10, nevts_with_jpsi_dstar_l10, nevts_with_jpsi_dstar_l10/nevts_with_jpsi_dstar_l9, nevts_with_jpsi_dstar_l10/nevts_with_jpsi_dstar_l0]
    writer.writerow(l10)
    
    l11 = [11, 'Kaon and Pion dz < 1 cm', JpsiDstar_l11, nevts_with_jpsi_dstar_l11, nevts_with_jpsi_dstar_l11/nevts_with_jpsi_dstar_l10, nevts_with_jpsi_dstar_l11/nevts_with_jpsi_dstar_l0]
    writer.writerow(l11)
    
    l12 = [12, 'Slow pion pT > 0.3 GeV', JpsiDstar_l12, nevts_with_jpsi_dstar_l12, nevts_with_jpsi_dstar_l12/nevts_with_jpsi_dstar_l11, nevts_with_jpsi_dstar_l12/nevts_with_jpsi_dstar_l0]
    writer.writerow(l12)
    
    l13 = [13, 'Slow pion track 𝜒²/ndof < 3', JpsiDstar_l13, nevts_with_jpsi_dstar_l13, nevts_with_jpsi_dstar_l13/nevts_with_jpsi_dstar_l12, nevts_with_jpsi_dstar_l13/nevts_with_jpsi_dstar_l0]
    writer.writerow(l13)
    
    l14 = [14, 'Slow pion valid hits > 2', JpsiDstar_l14, nevts_with_jpsi_dstar_l14, nevts_with_jpsi_dstar_l14/nevts_with_jpsi_dstar_l13, nevts_with_jpsi_dstar_l14/nevts_with_jpsi_dstar_l0]
    writer.writerow(l14)
    
    l15 = [15, 'D0 (from D*) cos(phi) > 0.99 (vertex alignment)', JpsiDstar_l15, nevts_with_jpsi_dstar_l15, nevts_with_jpsi_dstar_l15/nevts_with_jpsi_dstar_l14, nevts_with_jpsi_dstar_l15/nevts_with_jpsi_dstar_l0]
    writer.writerow(l15)
    
    l16 = [16, ' D0PDGmass - 0.028 < D0 (from D*) mass < D0PDGmass + 0.028', JpsiDstar_l16, nevts_with_jpsi_dstar_l16, nevts_with_jpsi_dstar_l16/nevts_with_jpsi_dstar_l15, nevts_with_jpsi_dstar_l16/nevts_with_jpsi_dstar_l0]
    writer.writerow(l16)
    
    l17 = [17, 'D0 (from D*) pT > 3 GeV', JpsiDstar_l17, nevts_with_jpsi_dstar_l17, nevts_with_jpsi_dstar_l17/nevts_with_jpsi_dstar_l16, nevts_with_jpsi_dstar_l17/nevts_with_jpsi_dstar_l0]
    writer.writerow(l17)
    
    l18 = [18, 'D0 (from D*) decay length significance > 3', JpsiDstar_l18, nevts_with_jpsi_dstar_l18, nevts_with_jpsi_dstar_l18/nevts_with_jpsi_dstar_l17, nevts_with_jpsi_dstar_l18/nevts_with_jpsi_dstar_l0]
    writer.writerow(l18)
    
    l19 = [19, 'D* pT > 3 GeV', JpsiDstar_l19, nevts_with_jpsi_dstar_l19, nevts_with_jpsi_dstar_l19/nevts_with_jpsi_dstar_l18, nevts_with_jpsi_dstar_l19/nevts_with_jpsi_dstar_l0]
    writer.writerow(l19)
    
    l20 = [20, 'D* right charge ', JpsiDstar_l20, nevts_with_jpsi_dstar_l20, nevts_with_jpsi_dstar_l20/nevts_with_jpsi_dstar_l19, nevts_with_jpsi_dstar_l20/nevts_with_jpsi_dstar_l0]
    writer.writerow(l20)
    
    jpsi_dstar_cuts = ['J/Ψ + D* cuts']
    writer.writerow(jpsi_dstar_cuts)
       
    l21 = [21, 'Association criteria: same vertex', JpsiDstar_l21, nevts_with_jpsi_dstar_l21, nevts_with_jpsi_dstar_l21/nevts_with_jpsi_dstar_l20, nevts_with_jpsi_dstar_l21/nevts_with_jpsi_dstar_l0]
    writer.writerow(l21)
    
    last_line = ['Global cuts', 'drop', JpsiDstar_l21/JpsiDstar_l0, nevts_with_jpsi_dstar_l21/nevts_with_jpsi_dstar_l0]
    writer.writerow(last_line)
    
    